In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = "C:\\Users\\cm\\Desktop\\deployed-\\30052021 Housing data Kaggle\\8. Pipline\\future selection\\X_train_notscaled.csv"

In [3]:
X = pd.read_csv(path)

In [4]:
X.head()

,MSSubClass,LotFrontage,LotArea,Neighborhood,OverallQual,YearBuilt,YearRemodAdd,ExterQual,BsmtQual,TotalBsmtSF,GrLivArea,FullBath,KitchenQual,GarageFinish,GarageCars
0,60,4.174387,9.041922,0.102740,7,2003,2003,0.334247,0.423288,856,7.444249,2,0.401370,0.289041,2
1,20,4.382027,9.169518,0.007534,6,1976,1976,0.620548,0.423288,1262,7.140453,2,0.503425,0.289041,2
2,60,4.219508,9.328123,0.102740,7,2001,2002,0.334247,0.423288,920,7.487734,2,0.401370,0.289041,2
3,70,4.094345,9.164296,0.034932,7,1915,1970,0.620548,0.444521,756,7.448334,1,0.401370,0.414384,3
4,60,4.430817,9.565214,0.028082,8,2000,2000,0.334247,0.423288,1145,7.695303,2,0.401370,0.289041,3


In [5]:
X.shape

(1460, 15)

In [6]:
ypath = "C:\\Users\\cm\\Desktop\\deployed-\\30052021 Housing data Kaggle\\8. Pipline\\future selection\\y_train_notscaled.csv"

In [7]:
y = pd.read_csv(ypath)

In [8]:
y.shape

(1460, 1)

In [9]:
y.head()

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216


In [10]:
# spliting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [11]:

# Training the creating base model on the  dataset
from sklearn.ensemble import RandomForestRegressor
baseModel = RandomForestRegressor(random_state = 0)
baseModel.fit(X_train, y_train)


<ipython-input-11-1595f4c9d7f5>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  baseModel.fit(X_train, y_train)


RandomForestRegressor(random_state=0)

In [12]:
y_pred = baseModel.predict(X_test)

In [13]:
# evaluating base model
from sklearn.metrics import r2_score
score = r2_score(y_test, y_pred)

In [14]:
score

0.8736760760623905

base model has an accuracy of 87.3 %

**Tuning the model to find best number of trees**

In [15]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [16]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\cm\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [17]:
#Best number of trees
rf_random.best_estimator_

RandomForestRegressor(max_depth=100, max_features='sqrt', min_samples_split=5,
                      n_estimators=2000)

RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_split=5,
                      n_estimators=400)

In [18]:
y_hat = rf_random.predict(X_test)

In [19]:
score2 = r2_score(y_test, y_hat)

In [20]:
score2

0.8838742572998874

In [21]:
print('Improvement of {:0.2f}%.'.format( 100 * (score2 - score) / score))

Improvement of 1.17%.


Improvement of 1.13%.


base model has an accuracy of 87.3 %
<br>tuned model imporved accuracy to 88.3 %

This model has more accuracy compared to other models

In [22]:
#savinf models to local disk
import pickle

In [23]:
pickle.dump(rf_random, open("model.pkl", "wb"))

In [26]:
model = pickle.load(open("model.pkl", 'rb'))

In [28]:
model

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [29]:
print(model.predict(X_test))

[12.35579994 11.80892358 11.69998568 12.25377341 11.41797694 11.64992874
 12.38334056 11.65090743 13.05937301 11.97752407 12.25120178 11.9952845
 12.32087298 11.69616486 11.728245   11.81526372 12.3347402  11.61926447
 11.88776478 12.00044884 11.78742042 11.87193939 11.67531087 12.07055203
 12.15559555 12.27916457 12.05208183 11.29277048 12.69990826 11.66914469
 11.8827054  12.19117978 11.83255231 12.56990114 12.80008818 12.10043873
 12.54946823 11.75407784 12.33359722 12.61710224 12.2339182  11.66275653
 12.14683161 12.66585799 12.73979172 11.86747988 11.73089658 11.73493701
 12.05224112 11.48948043 12.85149713 11.90067151 12.03718091 11.31988213
 12.29338931 11.70921164 11.77646175 12.39572535 11.85240421 11.5086541
 11.88872588 11.7916518  11.88919869 11.94567668 12.16805212 11.94921912
 11.80809175 12.30715971 11.70784978 12.20452387 12.08329962 11.63231028
 11.46764621 12.27135682 11.59608973 12.50007251 11.77603865 11.56622814
 12.49284358 11.95644358 11.77797204 11.7430825  11.6